아이템 기반 필터링 구현하기

- 사용자 선호에 따른 장르, 감독별, 줄거리 유사도 검증 구현 

In [20]:
##########################################

# 비회원 장르별 순위 추천


In [21]:
# 1. 데이터 가져오기 
# 1-1. import 설정
import pymysql
import pandas as pd


#1-2. DB에서 데이터 가져오기
# inner join을 이용해 movie, genre 테이블 가져오기

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
cursor = conn.cursor()

# sql 가져오기
sql = """
    select m.*, g.*, d.*, a.*
    from movie m
    left outer join moviegenre mg
    on m.moviecd = mg.moviecd
    left outer join genre g
    on mg.genrecd = g.genrecd
    left outer join moviedirector md
    on m.moviecd = md.moviecd
    left outer join director d
    on md.directorcd = d.directorcd
    left outer join movieactor ma
    on m.moviecd = ma.moviecd
    left outer join actor a
    on ma.actorcd = a.actorcd
"""

# sql 실행
cursor.execute(query=sql)
rows = cursor.fetchall()

# 결과 가져오기
# for row in rows:
#     print(row)

# 데이터프레임으로 변환하기
movie_content = pd.DataFrame(rows, columns=['movieCd', 'movieNm', 'movieNmEn', 'shortContent', 'prdtYear', 'showTm', 'openDt', 'naverScore', 'imdbScore', 'watchGradeCd', 'genreCd', 'genreNm', 'directorCd', 'directorNm', 'directorNmEn', 'actorCd', 'actorNm', 'actorNmEn'])
movie_content

,movieCd,movieNm,movieNmEn,shortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,genreCd,genreNm,directorCd,directorNm,directorNmEn,actorCd,actorNm,actorNmEn
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,1408,헨리 월설,None
1,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,7525,랄프 루이스,None
2,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,19081,D.W. 그리피스,None,13797,릴리안 기쉬,None
3,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,42546,D.W. 그리피스,None,1408,헨리 월설,None
4,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,10,전쟁,42546,D.W. 그리피스,None,7525,랄프 루이스,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520821,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,7,코미디,40137,최기윤,None,1,,None
520822,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,16672,최기윤,None,1,,None
520823,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,19,드라마,40137,최기윤,None,1,,None
520824,20219995,이멀전씨 키트,Emergency Kit,None,2021,None,0,None,None,None,4,기타,16747,전건우,None,1,,None


In [22]:
# 2. 데이터 전처리하기
# 2-1. shortContent, genreNm, directNm별로 전처리

movie_content1 = movie_content[['movieCd', 'movieNm', 'actorNm']]
movie_content2 = movie_content[['movieCd', 'movieNm', 'genreNm']]
movie_content3 = movie_content[['movieCd', 'movieNm', 'directorNm']]
movie_content4 = movie_content[['movieCd', 'movieNm', 'shortContent']]

#2-2. 결측치 ""로 대체하기
movie_content1["actorNm"] = movie_content1['actorNm'].fillna("")
movie_content2["genreNm"] = movie_content2['genreNm'].fillna("")
movie_content3["directorNm"] = movie_content3['directorNm'].fillna("")
movie_content4["shortContent"] = movie_content4['shortContent'].fillna("")

C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a

In [23]:
# 2-3. 유사도 비교 컬럼 전처리

import numpy as np

# actor
movie_content1["actorNm"] = movie_content1.groupby(by=["movieCd", "movieNm"])["actorNm"].transform(lambda row: '$$'.join(row))
movie_content1.drop_duplicates(inplace=True)
movie_content1['actorNm'] = movie_content1['actorNm'].map(lambda sc : ','.join(np.unique(sc.split('$$'))))
movie_content1.shape

# genre
movie_content2["genreNm"] = movie_content2.groupby(by=["movieCd", "movieNm"])["genreNm"].transform(lambda row: ','.join(row))
movie_content2.drop_duplicates(inplace=True)
movie_content2['genreNm'] = movie_content2['genreNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content2.shape


# director
movie_content3["directorNm"] = movie_content3.groupby(by=["movieCd", "movieNm"])["directorNm"].transform(lambda row: ','.join(row))
movie_content3.drop_duplicates(inplace=True)
movie_content3['directorNm'] = movie_content3['directorNm'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content3.shape

# shortContent
movie_content4["shortContent"] = movie_content4.groupby(by=["movieCd", "movieNm"])["shortContent"].transform(lambda row: ','.join(row))
movie_content4.drop_duplicates(inplace=True)
movie_content4['shortContent'] = movie_content4['shortContent'].map(lambda sc : ','.join(np.unique(sc.split(','))))
movie_content4.shape


C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

(46918, 3)

In [24]:
# 2-4. movieCd, movieNm 기준으로 통합하기
movie_contents_merge = movie_content1.merge(movie_content2, on=['movieCd', 'movieNm'], how="inner")\
                                        .merge(movie_content3, on=['movieCd', 'movieNm'], how="inner")\
                                            .merge(movie_content4, on=['movieCd', 'movieNm'], how="inner")

# 통합 데이터 체크
print( movie_contents_merge.shape )
movie_contents_merge.head()

(46918, 6)


,movieCd,movieNm,actorNm,genreNm,directorNm,shortContent
0,19158001,국가의 탄생,"랄프 루이스,릴리안 기쉬,헨리 월설","드라마,서부극(웨스턴),전쟁",D.W. 그리피스,
1,19188001,개의 일생,찰리 채플린,코미디,찰리 채플린,
2,19190002,여름밤의 기묘한 이야기,"아니타 베르버,콘라드 바이트",공포(호러),리차드 오스왈드,
3,19190003,재판장,"리차드 크리스텐슨,할바드 호프",드라마,칼 테오도르 드레이어,
4,19190007,의리적 구토,"김도산,윤혁,이경환","드라마,사극,액션",김도산,


In [25]:
# 2-5. content 컬럼 생성 
# shortContent + genreNm + directorNm 통합

movie_contents_merge['content'] = movie_contents_merge["actorNm"] + movie_contents_merge["genreNm"] + movie_contents_merge["directorNm"] + movie_contents_merge["shortContent"]

# 컬럼별 ','으로 구분하여 통합시키기
colums = ["actorNm", "genreNm", "directorNm", "shortContent"]
movie_contents_merge['content'] = movie_contents_merge[colums].apply(lambda row: ','.join(row.values), axis=1)
movie_contents_merge

,movieCd,movieNm,actorNm,genreNm,directorNm,shortContent,content
0,19158001,국가의 탄생,"랄프 루이스,릴리안 기쉬,헨리 월설","드라마,서부극(웨스턴),전쟁",D.W. 그리피스,,"랄프 루이스,릴리안 기쉬,헨리 월설,드라마,서부극(웨스턴),전쟁,D.W. 그리피스,"
1,19188001,개의 일생,찰리 채플린,코미디,찰리 채플린,,"찰리 채플린,코미디,찰리 채플린,"
2,19190002,여름밤의 기묘한 이야기,"아니타 베르버,콘라드 바이트",공포(호러),리차드 오스왈드,,"아니타 베르버,콘라드 바이트,공포(호러),리차드 오스왈드,"
3,19190003,재판장,"리차드 크리스텐슨,할바드 호프",드라마,칼 테오도르 드레이어,,"리차드 크리스텐슨,할바드 호프,드라마,칼 테오도르 드레이어,"
4,19190007,의리적 구토,"김도산,윤혁,이경환","드라마,사극,액션",김도산,,"김도산,윤혁,이경환,드라마,사극,액션,김도산,"
...,...,...,...,...,...,...,...
46913,20219964,생각 끝에서 찾아오는,"김지경,주수정",드라마,이수학,,"김지경,주수정,드라마,이수학,"
46914,20219978,볼레로 만들기,,기타,이와,,",기타,이와,"
46915,20219984,어디에도 없는 시간,,드라마,장태구,,",드라마,장태구,"
46916,20219993,청수탕 사람들,,"드라마,코미디",최기윤,,",드라마,코미디,최기윤,"


In [26]:
# 2-6. 행 범위 설정하기
# 데이터가 너무 많아 램 32gb로 계산할 수 없음
# 가능 범위 안으로 자르는 작업 수행
movie_contents_merge = movie_contents_merge.iloc[0:8000]

In [27]:
# 3. 문자열 genreNm 칼럼을 Count 기반으로 피처 벡터화 변환
# CountVectorizer = 단어 벡터화 카운트 정렬
# TfidfTransformer = TF-IDF로 단어 벡터화

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,1))
merge_content = count_vect.fit_transform(movie_contents_merge['content'])

tfidf_transformer = TfidfTransformer()

merge_content = tfidf_transformer.fit_transform(merge_content)

print(merge_content.shape)

(8000, 26236)


In [28]:
# 4. 리스트 객체 문자열 변경, count 피처 벡터화하기

from sklearn.metrics.pairwise import cosine_similarity

director_content_sim = cosine_similarity(merge_content, merge_content)
director_content_sim[0]


array([1., 0., 0., ..., 0., 0., 0.])

In [29]:
# 5. 특정영화 정보, 유사도 뽑아내기

def find_genre_score(movie_contents_merge, sim_metrics, title_name, top_n=100):

    # 입력한 영화 인덱스
    movieNm = movie_contents_merge[movie_contents_merge['movieNm'] == title_name]
    movieNm_index = movieNm.index.values

    # 입력한 영화 유사도 데이터 프레임을 추가하기
    movie_contents_merge["similarity"] = sim_metrics[movieNm_index, :].reshape(-1,1)

    # 유사도 내림차순 정렬(상위 인덱스 100개 추출)
    temp = movie_contents_merge.sort_values(by="similarity", ascending=False)
    final_index = temp.index.values[ : top_n]

    return movie_contents_merge.iloc[final_index]

In [30]:
# 6. 특정영화 유사한 장르 영화를 평점 순위별로 추천하기

# 6-1. 라이언 일병 구하기와 높은 유사도를 가지는 영화 순위별 정렬하기
similar_movie = find_genre_score(movie_contents_merge, director_content_sim, '라이언 일병 구하기', 30)
similar_movie = similar_movie[['movieCd', 'movieNm', 'similarity', 'content']]
similar_movie





C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,movieCd,movieNm,similarity,content
6998,19980211,라이언 일병 구하기,1.000000,"맷 데이먼,에드워드 번즈,톰 행크스,드라마,액션,전쟁,스티븐 스필버그, 2차 대전이..."
5042,19930396,시애틀의 잠 못이루는 밤,0.143664,"맥 라이언,톰 행크스,드라마,멜로/로맨스,노라 에프론,"
7066,19980281,유브 갓 메일,0.141175,"맥 라이언,톰 행크스,멜로/로맨스,코미디,노라 에프론,"
4783,19930124,도어즈,0.119928,"맥 라이언,발 킬머,드라마,올리버 스톤,"
2592,19830127,브레드레스,0.112761,"로버트 던,리차드 기어,존 P. 라이언,드라마,범죄,액션,짐 맥브라이드,"
6513,19970189,애딕티드 러브,0.106177,"매튜 브로데릭,맥 라이언,멜로/로맨스,코미디,그리핀 던,"
5108,19938761,악몽,0.104007,"기네스 팰트로우,데니스 퀘이드,맥 라이언,제임스 칸,드라마,미스터리,스티브 클로브스,"
5369,19940287,남자가 사랑할 때,0.102703,"맥 라이언,앤디 가르시아,드라마,멜로/로맨스,루이스 만도키,"
4737,19930077,키스의 전주곡,0.102449,"맥 라이언,알렉 볼드윈,멜로/로맨스,노만 르네,"
6199,19960275,커리지 언더 파이어,0.102363,"덴젤 워싱턴,루 다이아몬드 필립스,맥 라이언,드라마,미스터리,액션,전쟁,에드워드 즈윅,"
